In [22]:
from datetime import datetime as dt
from dateutil.relativedelta import relativedelta as rd
from concurrent.futures import ThreadPoolExecutor
import ast
import time

from google.colab import userdata
import requests, pandas as pd
from sklearn.cluster import affinity_propagation
from sklearn.preprocessing import MinMaxScaler

class History:
    PERIODS = (3, 8, 21, 55)

    def __init__(self):
        self.get_etf_items()
        self.get_prices()
        self.get_clusters()
        self.get_scores()

    def get_etf_items(self):
        URL='https://finance.naver.com/api/sise/etfItemList.nhn'
        data = requests.get(URL).json()\
            .get('result').get('etfItemList')
        etf_items = pd.DataFrame(data).dropna()\
            .loc[:, ['itemcode', 'etfTabCode', 'itemname',
                     'amonut', 'marketSum']]
        kwds = ['합성', '액티브', 'TR', '혼합', 'ESG',
                'BBIG', 'HANARO', 'ARIRANG', '닥100', 'P500',
                '\d\d-\d\d', '금리', '3년', '단기',
                '레버리지', '2X', '프리미엄', '숏', '커버드콜', '인도']
        expr = f'amonut > {etf_items.amonut.median() * 2} '\
               'and etfTabCode != 1 '\
               f'and marketSum > {etf_items.marketSum.median() * 2} '\
               f'and (not itemname.str.contains("{"|".join(kwds)}") '\
               'or (itemname.str.contains("인버스") '\
               'and itemname.str.contains("합성")))'
        etf_items.query(expr, inplace=True)
        group_kwds = ['SOLACTIVE', 'INDXX', 'Fn', 'KIS', '\(H\)',
                      '선물', '주$', 'iSelect', 'KRX', 'Plus', '플러스',
                      'TOP10', '200', '소부장', '산업', '소재', '테마',
                      '핵심\S*', 'AI']
        etf_items['group'] = etf_items.itemname\
            .replace('^(\S)+ ', '', regex=True)\
            .replace('|'.join(group_kwds), '', regex=True)\
            .replace('국고채', '국채', regex=True)\
            .str.strip()

        self.etf_items = etf_items.query('group != ""')\
            .groupby('group').first()\
            .drop('etfTabCode', axis=1)\
            .sort_values('marketSum', ascending=False)\
            .reset_index(drop=True).loc[:, ['itemcode', 'itemname']]

    @classmethod
    def get_price(cls, itemcode):
        URL = 'https://api.finance.naver.com/siseJson.naver'
        st = dt.utcnow() - rd(months=3)
        r = requests.get(URL, params=dict(symbol=itemcode, timeframe='day',
            requestType=1, startTime=st.strftime('%Y%m%d'), endTime='20991231'))
        data = ast.literal_eval(r.text.replace('\n', '').replace('\t', ''))
        df = pd.DataFrame(data[1:], columns=data[0])\
            .loc[:, ['고가', '저가', '종가']].tail(max(cls.PERIODS))
        return (itemcode, df)

    def get_prices(self):
        t = time.time()
        print('get_prices : start')
        with ThreadPoolExecutor() as excutor:
            result = excutor.map(
                self.get_price, self.etf_items.itemcode)
        self.prices = {}
        for symbol, data in result:
            if len(data) == max(self.PERIODS):
                self.prices[symbol] = data
        print(f'get_prices : finish ({time.time() - t:.2f}s)')

    @staticmethod
    def get_tr(price):
        concat = lambda x, y: pd.concat([x, y], axis=1)
        th = concat(price.종가.shift(1), price.고가).max(axis=1)
        tr = concat(price.종가.shift(1), price.저가).min(axis=1)
        return (th - tr).reset_index(drop=True)

    @property
    def volatility(self):
        return {symbol : self.get_tr(price)
            for symbol, price in self.prices.items()}

    @property
    def corr_matrix(self):
        df = pd.concat(self.volatility.values(), axis=1)
        scaled = MinMaxScaler().fit_transform(df)
        df_ = pd.DataFrame(scaled)
        df_.columns = self.volatility.keys()
        return df_.corr()

    def get_clusters(self):
        _, groups = affinity_propagation(self.corr_matrix)
        self.clusters = pd.DataFrame(
            zip(self.corr_matrix.index, groups),
            columns=['Symbol', 'Group'])\
            .merge(self.etf_items,
                   left_on='Symbol', right_on='itemcode')\
            .loc[:, ['Symbol', 'Group', 'itemname']]

    def get_momentum(self, symbol):
        close = self.prices.get(symbol).종가
        return sum([close.iloc[-1] / close.iloc[-p] - 1
                    for p in self.PERIODS]) / len(self.PERIODS)

    def get_risk(self, symbol):
        return self.volatility.get(symbol)\
            .ewm(max(self.PERIODS)).mean().iloc[-1]\
            / self.prices.get(symbol).종가.iloc[-1]

    def get_scores(self):
        clusters_ = self.clusters.copy()
        clusters_['Momentum'] = clusters_.Symbol.apply(self.get_momentum)
        clusters_['Risk'] = clusters_.Symbol.apply(self.get_risk)
        self.scores = clusters_.loc[
            clusters_.groupby('Group').Momentum.idxmax()]\
            .sort_values('Momentum', ascending=False)\
            .reset_index(drop=True)

class Util:
    @staticmethod
    def check_expired_token(expired):
        dt_fmt = '%Y-%m-%d %H:%M:%S'
        now = dt.utcnow() + rd(hours=9)
        expired_dt = dt.strptime(expired, dt_fmt) - rd(hours=1)
        return expired_dt < now

    @staticmethod
    def get_value(item, key):
        return item.get(key, {}).get('S')

    @staticmethod
    def get_api(URL, PATH='/', params={}, headers={}):
        return requests.get(URL + PATH,
            params=params, headers=headers).json()

    @staticmethod
    def post_api(URL, PATH='/', payload={}, headers={}):
        return requests.post(URL + PATH,
            json=payload, headers=headers).json()

class Biz:
    KIS_DOMAIN = 'https://openapi.koreainvestment.com:9443'
    KIS_CANO = userdata.get('KIS_CANO')
    KIS_APPKEY = userdata.get('KIS_APPKEY')
    KIS_APPSECRET = userdata.get('KIS_APPSECRET')
    KIS_TRADING = '/uapi/domestic-stock/v1/trading'
    API_DOMAIN = userdata.get('KIS_AUTH_TOKEN_API')
    DIV = 4
    TARGET_RISK = 0.01

    def __init__(self):
        self.history = History()
        self.get_token()
        self.get_account_balance()
        self.get_balance()
        self.get_exit()
        self.get_enter()
        self.info()
        self.trade()

    def info(self):
        print('\nACCOUNT')
        print(f'₩{self.account_balance:,}')
        if len(self.balance):
            print('\nBALANCE')
            print(self.balance)
        if len(self.exit):
            print('\nEXIT')
            print(self.exit)
        if len(self.enter):
            print('\nENTER')
            print(self.enter)

    @property
    def headers(self):
        return {
            'content-type': 'application/json; charset=utf-8',
            'authorization': f'Bearer {self.token}',
            'appkey': self.KIS_APPKEY,
            'appsecret': self.KIS_APPSECRET,
            'custtype': 'P',
        }

    @property
    def params(self):
        return dict(
            CANO=self.KIS_CANO,
            ACNT_PRDT_CD='01',
            INQR_DVSN_1='',
            BSPR_BF_DT_APLY_YN='')

    @classmethod
    def get_token_from_server(cls) -> tuple:
        PATH = '/oauth2/tokenP'
        payload = dict(
            grant_type='client_credentials',
            appkey=cls.KIS_APPKEY,
            appsecret=cls.KIS_APPSECRET,
        )

        json = Util.post_api(cls.KIS_DOMAIN, PATH,
                payload=payload)
        return (json.get('access_token'),
                json.get('access_token_token_expired'))

    @classmethod
    def save_token(cls, token: str, expired: str) -> tuple:
        payload = dict(
            cano={'S': cls.KIS_CANO},
            token={'S': token},
            expired={'S': expired},
        )

        msg = Util.post_api(cls.API_DOMAIN,
                payload=payload)
        if msg: print(msg)

    @classmethod
    def trading_path(cls, path):
        return cls.KIS_TRADING + path

    def get_token(self):
        params = dict(cano=self.KIS_CANO)
        json = Util.get_api(self.API_DOMAIN,
                params=params)
        item = json.get('Item', {})
        try:
            token = Util.get_value(item, 'token')
            if not token:
                raise Exception('저장된 토큰 없음')
            expired = Util.get_value(item, 'expired')
            if Util.check_expired_token(expired):
                raise Exception('만료된 토큰')
        except Exception as e:
            print(e)
            token, expired = self.get_token_from_server()
            self.save_token(token, expired)
        self.token : str = token

    def get_account_balance(self):
        PATH = self.trading_path('/inquire-account-balance')
        headers = self.headers
        headers['tr_id'] = 'CTRP6548R'
        json = Util.get_api(self.KIS_DOMAIN, PATH,
            params=self.params, headers=headers)
        df = pd.DataFrame(json.get('output1')[:-1])
        df.columns=['매입금액', '평가금액', '평가손익금액',
            '신용대출금액', '실제순자산금액', '전체비중율']
        df.index = [
            '주식', '펀드/MMW', '채권', 'ELS/DLS', 'WRAP', '신탁/퇴직연금/외화신탁',
            'RP/발행어음', '해외주식', '해외채권', '금현물', 'CD/CP', '단기사채', '타사상품',
            '외화단기사채', '외화 ELS/DLS', '외화', '예수금+CMA', '청약자예수금']
        self.account_balance = df.loc[:, "평가금액"].drop('채권').astype(int).sum()

    def get_balance(self):
        PATH = self.trading_path('/inquire-balance')
        # params
        params = self.params
        params['AFHR_FLPR_YN'] = 'N'
        params['OFL_YN'] = ''
        params['INQR_DVSN'] = '02'
        params['UNPR_DVSN'] = '01'
        params['FUND_STTL_ICLD_YN'] = 'N'
        params['FNCG_AMT_AUTO_RDPT_YN'] = 'N'
        params['PRCS_DVSN'] = '00'
        params['CTX_AREA_FK100'] = ''
        params['CTX_AREA_NK100'] = ''
        # headers
        headers = self.headers
        headers['tr_id'] = 'TTTC8434R'
        # data
        json = Util.get_api(self.KIS_DOMAIN, PATH,
            params=params, headers=headers)
        df = pd.DataFrame(json.get('output1'),
            columns=['pdno', 'prdt_name', 'hldg_qty'])\
            .astype({'hldg_qty': int})
        df.query('hldg_qty > 0', inplace=True)
        df.columns = ['상품번호', '상품명', '보유수량']
        df.set_index('상품번호', inplace=True)
        self.balance = df.copy()

    def get_exit(self):
        df = self.balance.merge(
            self.history.scores.head(5),
            left_index=True, right_on='Symbol', how='left')
        self.exit = df[df.itemname.isna()]\
            .loc[:,['Symbol', '상품명', '보유수량']].set_index('Symbol')

    def get_enter(self):
        df = self.balance.merge(
            self.history.scores
                .query('Momentum > 0')
                .head(self.DIV + 1),
            left_index=True, right_on='Symbol', how='right')
        # print(df)
        df = df[df.상품명.isna()]\
            .head(len(self.exit) + (self.DIV - len(self.balance)))\
            .loc[:,['Symbol', 'itemname', 'Risk']]
        df['Price'] = df.Symbol.apply(
            lambda x: self.history.prices.get(x).종가.iloc[-1])
        self.enter = df.set_index('Symbol')

    def trade(self):
        for idx, row in self.exit.iterrows():
            print(f'\n{row["상품명"]}({idx}) : 매도')
            self.sell(idx, row['보유수량'])
        for idx, row in self.enter.iterrows():
            print(f'\n{row["itemname"]}({idx}) : 매수')
            amt = int(min(1, self.TARGET_RISK / row['Risk'])
                    * self.account_balance / self.DIV / 10000) * 10000
            self.buy(idx, amt, row['Price'])

    def sell(self, pdno, qty):
        PATH = self.trading_path('/order-cash')
        # payload
        payload = {
            'CANO': self.KIS_CANO,
            'ACNT_PRDT_CD': '01',
            'PDNO': pdno,
            'ORD_DVSN': '01',
            'ORD_QTY': f'{qty}',
            'ORD_UNPR': '0'
        }
        # headers
        headers = self.headers
        headers['tr_id'] = 'TTTC0801U'
        # msg
        msg = Util.post_api(self.KIS_DOMAIN, PATH,
            payload=payload, headers=headers)
        print(msg.get('msg1'))

    def buy(self, pdno, amt, price):
        PATH = self.trading_path('/order-cash')
        qty = int(amt // price)
        # payload
        payload = {
            'CANO': self.KIS_CANO,
            'ACNT_PRDT_CD': '01',
            'PDNO': pdno,
            'ORD_DVSN': '01',
            'ORD_QTY': f'{qty}',
            'ORD_UNPR': '0'
        }
        # headers
        headers = self.headers
        headers['tr_id'] = 'TTTC0802U'
        # msg
        msg = Util.post_api(self.KIS_DOMAIN, PATH,
            payload=payload, headers=headers)
        print(msg.get('msg1'))

biz = Biz()

get_prices : start
get_prices : finish (14.21s)
   Symbol  Group                  itemname  Momentum      Risk
0  385560      2  KBSTAR KIS국고채30년Enhanced  0.108721  0.014568
1  300640      5               KBSTAR 게임테마  0.084783  0.023699
2  371450      7     TIGER 글로벌클라우드컴퓨팅INDXX  0.081913  0.014417
3  143860      0                TIGER 헬스케어  0.070378  0.017422
4  381180      6       TIGER 미국필라델피아반도체나스닥  0.061480  0.012521
5  144600      3              KODEX 은선물(H)  0.052532  0.017777
6  309230      1         ACE 미국WideMoat가치주  0.043012  0.010092
7  365000      9            TIGER 인터넷TOP10  0.020078  0.022229
8  305080      4            TIGER 미국채10년선물  0.019459  0.008070

ACCOUNT
₩27,937,683

BALANCE
                             상품명  보유수량
상품번호                                  
143860                TIGER 헬스케어   140
300640               KBSTAR 게임테마   304
381180       TIGER 미국필라델피아반도체나스닥   411
385560  KBSTAR KIS국고채30년Enhanced    65
